<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-10-09 15:35:31--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G   137MB/s    in 11s     

2021-10-09 15:35:42 (129 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [4]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [5]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

Как мы видим, дословно слово 'cat' не входит, однако на 4ом месте стоит слово cats. 

In [6]:
wv_embeddings.most_similar('dog',topn=10)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398118972778),
 ('bird', 0.7315489053726196),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349938392639),
 ('labrador', 0.7209131717681885)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [65]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        words = re.findall('\w+', text)
        for i in range(len(words)):
            words[i] = words[i].lower()
        return words
tokenizer = MyTokenizer()

In [66]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    n = 0
    words = tokenizer.tokenize(question)
    question_vector = np.zeros_like(embeddings[embeddings.index2entity[0]])
    for word in words:
        try:
            embeddings[word]
        except:
            #print(f'Word {word} is not in list of embedidngs')
            continue
        question_vector += embeddings[word]
        n += 1

    if n == 0:
        return question_vector
        
    return question_vector / n

    
    '''your code'''

In [9]:
question_to_vec('I abirvalg', wv_embeddings, tokenizer)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [10]:
ans = question_to_vec('I love neural networks', wv_embeddings, tokenizer)
print(ans)

[-1.0142275  -1.6891261  -1.2854122  -1.3710302   0.15916634  0.06706961
  1.1188413  -0.07051762  0.5386369  -0.37139115  0.28913018  1.03125
 -2.865466   -1.3192873   2.2098296   1.1678672   1.0841951   1.8467226
 -0.98733807  0.8720946  -0.17219625  0.73839015 -0.7827826  -0.8827869
 -0.6152611  -0.15872358 -0.6170798   0.36668167  2.0137532   0.2627472
 -1.4950123  -1.386715   -1.2049937  -1.016617    2.7621174   1.7255713
 -0.19038178 -0.5899447  -0.1141155  -1.8890352  -1.7883505  -0.20420384
 -0.7062034   2.489034   -0.15452717  1.1255013   0.2610835   1.3153247
 -0.57667524 -0.3327141   1.6764902   0.91229993 -0.69880134  2.3752553
 -1.7634224   3.1378052   1.2243204  -1.0554415   0.7897571   0.04404024
  1.1623045  -0.4478023  -0.9149816   0.01501505 -0.85523087 -1.0183454
 -0.7162116   1.0727204   0.8881876  -0.29789424  1.4994873  -0.9384045
 -1.6828092   1.0525229   0.77871495 -3.34081     0.15682733  0.7449282
 -1.4001557   0.26698634 -0.73809963  0.37732854 -1.186255   -1

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [11]:
print(round(ans[2],2))

-1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

для максимума нам нужно, чтобы дубликаты не были на первом месте. Иначе будет задействоваться метрика DCG@1 что плохо. Но, благо у нас достаточно выбора, начиная от 2 до 47 места позиции дубликата в ранжировании. Поэтому сам максимум = 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

На самом деле это будет = 1/ log2(10)

In [12]:
round(1/np.log2(10),1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [67]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    hits_value = 0
    for rank in dup_ranks:
        if rank <= k:
            hits_value += 1
    return hits_value / len(dup_ranks)


In [68]:
hits_count([2,2,3,5],3)

0.75

In [69]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dcg_value = 0
    for rank in dup_ranks:
        if rank <= k:
            dcg_value += 1 / np.log2(1+rank)
        
    return dcg_value / len(dup_ranks)

In [70]:
dcg_score([2,2,3,5],3)

0.44046487678572877

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [71]:
import pandas as pd

In [72]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [73]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [20]:
!unzip drive/MyDrive/stackoverflow_similar_questions.zip

Archive:  drive/MyDrive/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [21]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [22]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [23]:
len(validation_data)

3760

Размер нескольких первых строк

In [24]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


In [25]:
validation_data[1]

['How to start PhoneStateListener programmatically?',
 'PhoneStateListener and service',
 'Java cast object[] to model',
 'WCF and What does this mean?',
 'How to uncheck checkbox using jQuery Uniform library',
 'Two projects with same code base',
 "Can't read php file when upload image to server in titanium",
 'create pandas dataframe from dictionary of dictionaries',
 'how to pass tkinter text to flex',
 'runtime error project source code not found and invalid string null exception?',
 'excel VBA new URL on existing IE',
 'Howto immediately update TextBox after correction of bound property?',
 'JAX-RS Jersey/Grizzly Define an interface resource',
 'JSONConvert.DeserializeObject not handling child array with unnamed array items',
 'Unexpected error while loading data',
 'Cucumber Capybara Selenium race condition with expect(all())',
 'How do I check if one vector is a subset of another?',
 'Dropdown list where options populate a second list js/php?',
 'Automated Email Sender',
 'mysql

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy
from copy import deepcopy

In [74]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):

    q_vec = question_to_vec(question, embeddings, tokenizer)
    candidates_vectors = [question_to_vec(candidate, embeddings, tokenizer) for candidate in candidates]
    similarities = []
    for vector in candidates_vectors:
        similarities.append(scipy.spatial.distance.cosine(q_vec, vector))
    sort_indexes = np.argsort(similarities)
    result = []
    for i in sort_indexes:
        result.append((i,candidates[i]))
    return result
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [75]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [76]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [('*', 'Getting all list items of an unordered list in PHP'), #скрыт
            ('*', 'select2 not displaying search results'), #скрыт
            ('*', 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

До того, как в токенайзер добавил .lower() было 102
после этого стало 021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [45]:
from tqdm.notebook import tqdm

In [77]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [47]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.878
DCG@1000: 0.444 | Hits@1000: 1.000


In [78]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

# это когда я додумался добавить .lower() в токенайзер

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.524 | Hits@  10: 0.650
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [50]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [79]:
words = []
for data in train_data:
    words.append(tokenizer.tokenize(' '.join(data)))

In [87]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [88]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [89]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.320 | Hits@   1: 0.320
DCG@   5: 0.389 | Hits@   5: 0.450
DCG@  10: 0.416 | Hits@  10: 0.535
DCG@ 100: 0.469 | Hits@ 100: 0.790
DCG@ 500: 0.489 | Hits@ 500: 0.942
DCG@1000: 0.495 | Hits@1000: 1.000


Window = 3
DCG@   1: 0.248 | Hits@   1: 0.248
DCG@   5: 0.307 | Hits@   5: 0.362
DCG@  10: 0.333 | Hits@  10: 0.442
DCG@ 100: 0.384 | Hits@ 100: 0.693
DCG@ 500: 0.412 | Hits@ 500: 0.910
DCG@1000: 0.421 | Hits@1000: 1.000

Window = 5
DCG@   1: 0.266 | Hits@   1: 0.266
DCG@   5: 0.331 | Hits@   5: 0.392
DCG@  10: 0.358 | Hits@  10: 0.473
DCG@ 100: 0.408 | Hits@ 100: 0.722
DCG@ 500: 0.434 | Hits@ 500: 0.928
DCG@1000: 0.442 | Hits@1000: 1.000

Window = 7
DCG@   1: 0.273 | Hits@   1: 0.273
DCG@   5: 0.342 | Hits@   5: 0.408
DCG@  10: 0.367 | Hits@  10: 0.485
DCG@ 100: 0.419 | Hits@ 100: 0.741
DCG@ 500: 0.444 | Hits@ 500: 0.935
DCG@1000: 0.450 | Hits@1000: 1.000

Window = 9
DCG@   1: 0.280 | Hits@   1: 0.280
DCG@   5: 0.349 | Hits@   5: 0.411
DCG@  10: 0.376 | Hits@  10: 0.496
DCG@ 100: 0.426 | Hits@ 100: 0.745
DCG@ 500: 0.451 | Hits@ 500: 0.939
DCG@1000: 0.457 | Hits@1000: 1.000

window = 11
DCG@   1: 0.279 | Hits@   1: 0.279
DCG@   5: 0.354 | Hits@   5: 0.420
DCG@  10: 0.380 | Hits@  10: 0.502
DCG@ 100: 0.430 | Hits@ 100: 0.755
DCG@ 500: 0.454 | Hits@ 500: 0.938
DCG@1000: 0.460 | Hits@1000: 1.000

Window = 13
DCG@   1: 0.287 | Hits@   1: 0.287
DCG@   5: 0.357 | Hits@   5: 0.416
DCG@  10: 0.383 | Hits@  10: 0.497
DCG@ 100: 0.434 | Hits@ 100: 0.750
DCG@ 500: 0.458 | Hits@ 500: 0.935
DCG@1000: 0.465 | Hits@1000: 1.000

Window = 15
DCG@   1: 0.287 | Hits@   1: 0.287
DCG@   5: 0.362 | Hits@   5: 0.428
DCG@  10: 0.385 | Hits@  10: 0.499
DCG@ 100: 0.439 | Hits@ 100: 0.764
DCG@ 500: 0.461 | Hits@ 500: 0.939
DCG@1000: 0.468 | Hits@1000: 1.

Window = 17
DCG@   1: 0.292 | Hits@   1: 0.292
DCG@   5: 0.367 | Hits@   5: 0.434
DCG@  10: 0.391 | Hits@  10: 0.507
DCG@ 100: 0.442 | Hits@ 100: 0.763
DCG@ 500: 0.464 | Hits@ 500: 0.936
DCG@1000: 0.471 | Hits@1000: 1.000

Window = 19
DCG@   1: 0.300 | Hits@   1: 0.300
DCG@   5: 0.371 | Hits@   5: 0.433
DCG@  10: 0.395 | Hits@  10: 0.506
DCG@ 100: 0.446 | Hits@ 100: 0.764
DCG@ 500: 0.468 | Hits@ 500: 0.937
DCG@1000: 0.475 | Hits@1000: 1.000

Window = 21
DCG@   1: 0.295 | Hits@   1: 0.295
DCG@   5: 0.369 | Hits@   5: 0.434
DCG@  10: 0.392 | Hits@  10: 0.505
DCG@ 100: 0.443 | Hits@ 100: 0.756
DCG@ 500: 0.466 | Hits@ 500: 0.933
DCG@1000: 0.473 | Hits@1000: 1.000

Как видим, 19 был оптимален, далее уже слишком большое окно

это все когда я не додумался поставить .lower() для него окно уже другое

при lower() получаем лучше результаты, но почему-то после обучения результаты хуже чем без обучения. При уже времени перебирать window не было так. Однако попробовал 5, 15, 21 и на 21 лучший результат был, однако все же хуже чем без обучения.

давайте попробуем добавить лемматизацию

In [90]:
words = []
for data in train_data:
    words.append(tokenizer.tokenize(' '.join(data)))

In [91]:
words[0]

['converting',
 'string',
 'to',
 'list',
 'convert',
 'google',
 'results',
 'object',
 'pure',
 'js',
 'to',
 'python',
 'object']

In [96]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [97]:
for i in range(len(words)):
    for j in range(len(words[i])):
        words[i][j] = lemmatizer.lemmatize(words[i][j])

In [100]:
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=19).wv

In [101]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [102]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.318 | Hits@   1: 0.318
DCG@   5: 0.392 | Hits@   5: 0.461
DCG@  10: 0.413 | Hits@  10: 0.527
DCG@ 100: 0.463 | Hits@ 100: 0.769
DCG@ 500: 0.484 | Hits@ 500: 0.932
DCG@1000: 0.491 | Hits@1000: 1.000


Довольно странно, но с лемматизацией вышло хуже качество чем без неё. Видимо это из-за неидеальности самого способа приведения слов к начальной форме. И за счет специфичности слов программистских.

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


 Лучше качество дало конечно же разбиение по словам, но при этом приведение их к одному регистру. Вообще часто слова в разных регистрах и вправду могут нести разный смысл. Но видимо в данном случае было важнее найти общее в этих словах так.
 как раз .lower() можно отнести в какой-то мере к нормализации слов. Однако та же лемматизация почему-то не особо помогла. Хотя ожидалось обратное, т.к. если больше слов будут менее отличаться между собой из-за различных окончаний, мы сможем найти общее между ними лучше.
 Думаю качество вышло плохим из-за большой специфичности программистских слов. Также сами эмбеддинги, которые мы использовали довольно простые. Можно было бы написать поверх всего более умную нейронную сеть, которая бы училась правильно классифицировать опираясь, например на вектора, полученные из слов или вопросов.
